In [ ]:
!pip install transformers
!pip install kss

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 4.9 MB/s 
     |████████████████████████████████| 7.6 MB 43.2 MB/s 
     |████████████████████████████████| 163 kB 88.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42.4 MB 1.3 MB/s 
     |████████████████████████████████| 131 kB 77.2 MB/s 
  Created wheel for kss: filename=kss-3.6.4-py3-none-any.whl size=42448613 sha256=6c0d4b62be6bec5682fe15e7ad09c2cc8ca6bcbce7f648269fa5482ff9f2f01e
  Stored in directory: /root/.cache/pip/wheels/32/53/7c/76bdf098c2a6c5cd4c4c29648da30ad4793d604314b2aeb26f
Successfully built kss


In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

LABELS = ['불평/불만',
 '환영/호의',
 '감동/감탄',
 '지긋지긋',
 '고마움',
 '슬픔',
 '화남/분노',
 '존경',
 '기대감',
 '우쭐댐/무시함',
 '안타까움/실망',
 '비장함',
 '의심/불신',
 '뿌듯함',
 '편안/쾌적',
 '신기함/관심',
 '아껴주는',
 '부끄러움',
 '공포/무서움',
 '절망',
 '한심함',
 '역겨움/징그러움',
 '짜증',
 '어이없음',
 '없음',
 '패배/자기혐오',
 '귀찮음',
 '힘듦/지침',
 '즐거움/신남',
 '깨달음',
 '죄책감',
 '증오/혐오',
 '흐뭇함(귀여움/예쁨)',
 '당황/난처',
 '경악',
 '부담/안_내킴',
 '서러움',
 '재미없음',
 '불쌍함/연민',
 '놀람',
 '행복',
 '불안/걱정',
 '기쁨',
 '안심/신뢰']

In [ ]:
import torch
import torch.nn as nn
from transformers import ElectraConfig, ElectraModel
import numpy as np


class ELECTRALSTMClassification(nn.Module):
    def __init__(self):
        super().__init__()
        self.device = 'cuda'
        self.config = ElectraConfig.from_pretrained("beomi/KcELECTRA-base",
                                                    problem_type="multi_label_classification",
                                                    num_labels = 44) 
        
        self.embedding_size = 768
        self.batch_size = 32

        self.electra = ElectraModel.from_pretrained("beomi/KcELECTRA-base",config=self.config).to(self.device)
        self.lstm = nn.LSTM(self.embedding_size, self.embedding_size, batch_first=True, bidirectional=True).to(self.device)
        self.fc1 = nn.Linear(self.embedding_size * 5, 44)
        self.fc2 = nn.Linear(self.embedding_size * 2, 44)
        self.gelu = nn.GELU()


    def forward(self, input_ids=None, attention_mask=None, sep_idx=None):
        
        electra_output = self.electra(input_ids, attention_mask)[0]

        cls = electra_output[:, 0, :] # <CLS> embeddings
        # sep 토큰 가져오기
        sep_idx_x = sep_idx[0]
        sep_idx_y = sep_idx[1]

        idx = 0
        cnt = 0
        longest = torch.where(sep_idx_x==torch.mode(sep_idx_x).values)[0].size()[0]
        # 초기화
        sep_embeddings = torch.zeros(cls.size(0), longest, self.embedding_size).to(self.device)

        # embedding 값 집어넣어주기
        for x, y in zip(sep_idx_x, sep_idx_y):
            if idx == x:
                sep_embeddings[x, cnt, :] += electra_output[x, y, :]
                cnt += 1
            else:
                idx += 1
                cnt = 0
                sep_embeddings[x, cnt, :] += electra_output[x, y, :]

        # lstm 실행
        lstm_output, (h, c) = self.lstm(sep_embeddings) # (batch_size, seq_length, embedding_size)

        # lstm 처음과 끝 가져오기
        sep_first = lstm_output[:, 0, :]
        sep_last = lstm_output[:, -1, :]

        # lstm 결과와 cls 토큰 합치기
        concats = torch.cat((cls, sep_first, sep_last), dim=1)
        # fc 레이어에 넣고 44개 output
        x = self.gelu(concats)
        output = self.fc1(x)

        first_output = self.fc2(sep_first)
        last_output = self.fc2(sep_last)

        
        return output, first_output, last_output

In [ ]:
def cos_similiarity(v1, v2):
    dot_product = np.dot(v1, v2)
    l2_norm = (np.sqrt(sum(np.square(v1)))*np.sqrt(sum(np.square(v2))))
    similarity = dot_product/l2_norm

    return similarity

In [ ]:
import kss

def kss_sentence(sent):
    x = ''
    split_sent = kss.split_sentences(sent)
    for i,s in enumerate(split_sent):
        if i == 0:
            x = s
        else:
            x += ' [SEP] ' + s
    return x

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('beomi/KcELECTRA-base', do_lower_case=False)

def embedding(text):
    embeddings = tokenizer(text,
                           truncation=True,
                           max_length=512,
                           padding="max_length",
                           return_token_type_ids=False,
                           return_attention_mask=True,
                           add_special_tokens=True)
    return embeddings

Downloading:   0%|          | 0.00/288 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/504 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/396k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
dic_ori = "졸업식이 취소되어서 너무 아쉽다. 취소 이유는 중국 우한에서 처음 발병한 신종 코로나 바이러스 때문이다. 인생 마지막 졸업식인데 이렇게 취소가 되어버리다니 너무 허무하다. 사실은 졸업식에 별로 가고 싶지 않았다. 강의실에 우르르 모여 앉아 형식적인 학생회장의 말을 듣고, 그저 강의하는 사람 그 이상 그 이하도 아니었던 교수님들과 어색한 포옹을 하고 악수를 하며 학위기를 받는 그 상황이 너무 민망하고 어색할 것 같아서 가고 싶은 마음이 없었다. 그러다 마음을 바꾸게 된 건 우리 엄마의 부탁 때문이었다. 원래대로라면 2년 전에 이미 졸업을 했었어야 하지만 휴학, 유예 등으로 졸업이 2년이나 밀렸다. 2년을 기다려주시고, 이제 50대가 된 우리 엄마가 '더 늙기 전에 학사모 쓰고 사진 찍고 싶어!'라며 귀여운 부탁을 했다. 끝까지 거절하려다가 정말 얼마전에 엄마한테 졸업식 가자고 말씀드렸고, 엄마도 졸업식 날에 맞춰 연차까지 쓰셨지만 타이밍도 참. 속상하다. 팔정도에 있는 코끼리는 졸업식 때에만 올라탈 수 있다는 소문이 있다. 처음엔 갈 생각 없었던 졸업식이지만, 갈 마음을 먹고 나니 학사모 쓰고 코끼리 올라 탈 생각에 설레었는데 졸업식이 아예 취소가 되어버리니까 아쉬운 마음이 드는 건 왜일까? 8월에 있을 가을 학위수여식과 함께 진행한다고 하는데, 어느 누가 그 때를 기억하고 가겠어. 나는 괜찮은데, 엄마가 너무 아쉬워해서 마음이 쓰인다. 대학 입학부터 졸업까지 아무탈 없이, 걱정 없이 학교 생활 할 수 있게 도와주신 부모님께 졸업식 날에 맞춰서 꽃다발 선물이라도 해드려야겠다는 생각을 했다. 부모님 덕분에 좋은 학교 다니며, 더 멋진 사람으로 성장할 수 있게 도와주신 것에 대한 감사의 표현을 하는 것은 당연한 것 아닌가? '조금만 더 열심히 할 걸' 아쉬움이 남는 4년의 학교생활이었지만, 부모님 곁을 떠나 처음으로 타지에서 혼자 생활을 해보는 것부터 시작해서 처음 경험해볼 수 있었던 것들이 많아 행복했다. 물론 행복만 한 건 아니었지만, 대학 생활이 좋은 기억으로만 남지는 않겠지만, 그래도 소중했던 4년의 시간 덕분에 스스로 성장할 수 있었다."
dic_data = kss_sentence(dic_ori)


movie_path = '/content/drive/MyDrive/final_project/영화_data.pkl'

movie_ori = pd.read_pickle(movie_path)

dic_emb = embedding(dic_data)


'\nmovie_emb = []\nfor i in range(10):\n    movie_emb.append(embedding(movie_data[i]))\n    '

In [ ]:
PATH = '/content/drive/MyDrive/final_project/data_processing/best_model.pth'

model = ELECTRALSTMClassification()
model.load_state_dict(torch.load(PATH)['model_state_dict'],strict=False)
model.to('cuda')

In [ ]:
input_id = torch.LongTensor(dic_emb['input_ids']).unsqueeze(0).to('cuda')
mask = torch.LongTensor(dic_emb['attention_mask']).unsqueeze(0).to('cuda')
sep_idx = torch.where(input_id == 3)
 
y_pred = model(input_id, mask, sep_idx)

In [ ]:
y = torch.sigmoid(y_pred[0])[0]
print(torch.sigmoid(y_pred[0]))
arr = np.zeros(44)
for i in range(44):
    if y.tolist()[i] >= 0.3:
        arr[i] = 1
    else:
        arr[i] = 0

tensor([[0.4097, 0.1800, 0.2688, 0.0665, 0.3218, 0.4670, 0.1843, 0.0474, 0.2244,
         0.0152, 0.7167, 0.0395, 0.1083, 0.1374, 0.1878, 0.0757, 0.2616, 0.0470,
         0.0407, 0.1805, 0.0549, 0.0118, 0.3170, 0.1485, 0.0281, 0.0748, 0.0706,
         0.3978, 0.1273, 0.2477, 0.0438, 0.0262, 0.0619, 0.4493, 0.0520, 0.0993,
         0.1582, 0.0323, 0.2463, 0.1549, 0.2268, 0.3294, 0.2683, 0.3836]],
       device='cuda:0', grad_fn=<SigmoidBackward0>)


In [ ]:
for l, p in zip(LABELS, arr):
    if p==1:
        print(f"{l}",end=' ')


for i in range(5000):
    if cos_similiarity(np.array(movie_ori['emotion'][i]),arr) >= 0.8:
        print('\n cosine : ',cos_similiarity(np.array(movie_ori['emotion'][i]),arr))
        print(movie_ori['제명'][i])
        for l, p in zip(LABELS, movie_ori['emotion'][i]):
            if p==1:
                print(f"{l}",end=' ')

불평/불만 고마움 슬픔 안타까움/실망 짜증 힘듦/지침 당황/난처 불안/걱정 안심/신뢰 
 cosine :  0.8249579113843053
라스트 씬
불평/불만 슬픔 안타까움/실망 짜증 힘듦/지침 당황/난처 불쌍함/연민 불안/걱정 
 cosine :  0.8249579113843053
지퍼스 크리퍼스 2
불평/불만 슬픔 화남/분노 안타까움/실망 짜증 힘듦/지침 당황/난처 불안/걱정 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.
